## Exercise: Classification of the CIFAR10 data using CNNs
### Author [Manas Bedmutha](https://github.com/manasbedmutha98)
The [CIFAR10](https://www.cs.toronto.edu/~kriz/cifar.html) has 32 x 32 x 3 (channels) images of 10 different categories which have been numbered as digits from 0 to 9. It is also one of the most common datasets for starting up with deep learning. Its download script comes inbuilt with the keras package.

This notebook will walk you through the developing a classification model for the dataset using a <b>Convolutional Neural Network</b> with the help of the <b>Sequenctial API</b> from Keras.

### Importing required libraries

In [1]:
import os

os.environ['KERAS_BACKEND'] = 'tensorflow'

import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
from matplotlib import pyplot as plt

from keras.layers import Input, Flatten, Dense, Dropout, MaxPooling2D
#from keras.layers.advanced_activations import LeakyReLU
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.convolutional import Conv2D

from keras.activations import *
from keras.optimizers import *
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.models import Model, Sequential

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Downloading the dataset

CIFAR10 comes as a part of the keras datasets. It contains 50,000 training images while 10,000 test images. Running the next cell will download it to your local systems.

In [2]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [3]:
X_train.shape, X_test.shape

((50000, 32, 32, 3), (10000, 32, 32, 3))

### Data Preprocessing

- Since we are using a convolutional network we do not need to flatten the array into 1D.
- Normalize pixel values between -1 and 1 (and Input type should be float)
- There are 10 classes so in order to compute the cross entropy loss function we need to one-hot encoded vectors.
- The labels refer to the expected classification output for a given image. They are converted from singular to one-hot encoded values from 0 to 9. That is, if a given image corresponds to 5, its encoding will be [0,0,0,0,0,1,0,0,0,0]

In [4]:
X_train = (X_train.astype(np.float32) - 127.5)/127.5
X_test = (X_test.astype(np.float32) - 127.5)/127.5

X_train.shape, X_test.shape

((50000, 32, 32, 3), (10000, 32, 32, 3))

In [5]:
num_classes = 10

# convert class vectors to binary class matrices
y_train = to_categorical(y_train, num_classes)
#y_test = to_categorical(y_test, num_classes)

### Creating a Keras Model

Construct a model using the sequential API with the following instructions:

- Inputs are normalized using BatchNormaliation followed by a Dropout layer with a rate of 0.3
- Then add a 2D convolutional layer with a kernel of 3x3 with 16 filters
- Activate the output with a relu using an Activation layer separately
- Output from the convolutional layer goes through a MaxPooling layer of size 2
- Add another 2D convolutional layer but ensure that the shape remains 'same' as previous layer having 32 filters
- Output from the convolutional layer goes through a MaxPooling layer
- Then Flatten the output and add a Dropout layer with a rate of 0.3
- Connect the output to a Dense layer containing 100 neurons followed by a BatchNormalization 
- and finally connect to the output layer with an softmax activation function

Print the model summary to see the network

In [22]:
model = Sequential()
model.add(BatchNormalization(input_shape=(32,32,3)))
model.add(Dropout(0.3))

# Activation separate
model.add(Conv2D(32,kernel_size=3))
model.add(Activation('relu'))
model.add(MaxPooling2D(2, padding='valid',strides=2))

# Activation in Conv2D definition
model.add(Conv2D(64,kernel_size=3, padding='same',activation='sigmoid'))
model.add(MaxPooling2D(2,strides=2))
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(100,activation='relu'))
model.add(BatchNormalization())
model.add(Dense(num_classes,activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_11 (Batc (None, 32, 32, 3)         12        
_________________________________________________________________
dropout_9 (Dropout)          (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
activation_5 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 15, 15, 64)        18496     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 7, 7, 64)          0         
__________

The final model is compiled using an optimizer, a loss function and a metric for performance improvement. 
- The loss function is used to depict how far is the current model from the ideal answer
- The optimizer refers to the method that will be used to minimize the loss
- The metrics correspond to how we want to measure the performance of the network

In [23]:
model.compile(loss='categorical_crossentropy', optimizer=Adagrad(), metrics=['accuracy'])

### Training and testing

- Training is done using the function fit(). We train out network for 5 epochs.
- Testing is done using the predict() function. We can also use evaluate() but since we want to later generate a classificiation report, we will use the former


In [ ]:
model.fit(X_train,y_train, epochs=3)

Epoch 1/3
50000/50000 [==============================] - 102s 2ms/step - loss: 1.6136 - acc: 0.4198
Epoch 2/3
50000/50000 [==============================] - 124s 2ms/step - loss: 1.3768 - acc: 0.5059
Epoch 3/3
36096/50000 [====================>.........] - ETA: 43s - loss: 1.2933 - acc: 0.5390

In [19]:
y_check = model.predict(X_test)

y_pred = np.array([np.argmax(y_check[j]) for j in range(len(y_check))])
y_test

array([[3],
       [8],
       [8],
       ...,
       [5],
       [1],
       [7]])

In [20]:
print(confusion_matrix(y_test, y_pred))

[[516   2 196  30  15  20  12   4 195  10]
 [124 422  70  33  25  29  18  18 197  64]
 [ 52   0 643  43  80 103  43  16  20   0]
 [ 12   1 269 279  70 288  55  10  12   4]
 [ 27   1 348  56 389  71  55  39  14   0]
 [  9   0 213 122  50 561  20  17   8   0]
 [  8   1 252  65  99  50 507   9   8   1]
 [ 22   0 143  64 111 149  22 477   8   4]
 [109  14  70  16   8  35   7   6 731   4]
 [103  83  83  61  24  49  40  44 198 315]]


In [21]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.53      0.52      0.52      1000
          1       0.81      0.42      0.55      1000
          2       0.28      0.64      0.39      1000
          3       0.36      0.28      0.32      1000
          4       0.45      0.39      0.42      1000
          5       0.41      0.56      0.48      1000
          6       0.65      0.51      0.57      1000
          7       0.75      0.48      0.58      1000
          8       0.53      0.73      0.61      1000
          9       0.78      0.32      0.45      1000

avg / total       0.55      0.48      0.49     10000



## References:
1. [Keras' official example](https://github.com/keras-team/keras/blob/master/examples/mnist_mlp.py) on Github
2. [Documentation References](https://keras.io/) for more info about every function/layer